In [1]:
import functions as f
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import s3fs
import cftime
import intake

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))
path='escience2022/Antoine/ESA_SMOS_Arctic_Sea_Surface_Salinity/'

sat_files = f.read_satellite_data(num_years=10, path=path)
dat = xr.open_dataset(s3.open(sat_files[2011][0])).squeeze()
dat['y'] = dat['y']*1000
dat['x'] = dat['x']*1000
sss = dat['sss']


In [ ]:
f.plot_sss_sat(sss, levels=50)

In [ ]:
cat = col.search(source_id=['CESM2'], experiment_id=['historical'], table_id=['Amon'], variable_id=['tas'], member_id=['r1i1p1f1'])
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})

In [ ]:
dataset_list = list(dset_dict.keys())
dset = dset_dict[dataset_list[0]]
print(dset['tas'])

dset['tas'].plot()

In [ ]:
ctd = xr.open_dataset('3901598_prof.nc')
#print(ctd.variables)

ctd['LATITUDE']

In [ ]:
ctd['LONGITUDE']

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.NorthPolarStereo())
st = ctd.isel(ctd['LONGITUDE'] == -3.9, method='nearest')
#specific_station = [ctd['LONGITUDE'].sel(-3.9, method='nearest'), ctd['latitude'].sel(70, method='nearest')]
ax.scatter(ctd['LONGITUDE'][:], ctd['LATITUDE'][:], transform=ccrs.PlateCarree())
ax.scatter(specific_station[0], specific_station[1], c='r')
ax.coastlines()
ax.gridlines(draw_labels=True)